In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
#!pip install torch --index-url https://download.pytorch.org/whl/cu118

In [38]:
#!pip install -U langchain playwright html2text sentence_transformers faiss-gpu peft bitsandbytes trl

In [39]:
#!pip install -q accelerate==0.21.0

In [14]:

from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [15]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)

In [16]:
device = "cuda"

In [17]:
import torch
import transformers
from transformers import BitsAndBytesConfig

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 80:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [18]:
model_name = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [20]:
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config = bnb_config,
    do_sample=True,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [23]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [40]:
#!playwright install 
#!playwright install-deps 

In [25]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
articles = ["https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7015670/",
            "https://www.ncbi.nlm.nih.gov/books/NBK535404/",
            "https://www.kidneyfund.org/treatment-kidney-failure/",
            "https://www.niddk.nih.gov/health-information/kidney-disease/kidney-failure/choosing-treatment",
            "https://www.kidneyfund.org/treatments/medicines-manage-kidney-disease",
            "https://www.ucsf.edu/news/2023/08/426056/can-artificial-kidney-finally-free-patients-dialysis",
            "https://www.medscape.com/viewarticle/995477?form=fpf",
            "https://www.healio.com/news/nephrology/20231009/with-limited-funding-researchers-move-forward-on-bioartificial-kidneys-xenotransplants",
            "https://physicsworld.com/a/implantable-bioartificial-kidney-aims-to-free-patients-from-dialysis/",
            "https://www.freseniusmedicalcare.com/en/media/insights/company-features/the-history-of-dialysis",
            "https://www.telegraphindia.com/india/hope-for-artificial-kidney-as-scientists-announce-first-successful-animal-tests-with-implantable-device/cid/1962510",
            "https://www.genengnews.com/topics/translational-medicine/artificial-kidney-bioreactor-keeps-human-renal-cells-alive-in-pigs/",
            "https://medicine.yale.edu/news-article/dr-chang-wins-artificial-kidney-prize-phase-2-at-kidneyx-summit/",
           ]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [26]:
# Converts HTML to plain text 
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

In [27]:
# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=500, 
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

In [28]:
# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, 
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

In [29]:
prompt_template = """
### [INST] Instruction: Give answers backed by facts to the questons asked about kidney failure and artificial kidney. 
Here is the context: {context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [30]:
rag_chain = ( 
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What are the major symptoms of Kidney Disease?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [31]:
print(result['text'])


The major symptoms of kidney disease can vary depending on the stage of the disease and its underlying cause. However, some common symptoms of kidney disease include nausea, vomiting, loss of appetite, fatigue and weakness, sleep disturbance, oliguria (decreased urine output), decreased mental sharpness, muscle twitches and cramps, swelling of feet and ankles, persistent pruritus (itching), chest pain due to uremic pericarditis, arterial-venous nicking or retinopathy on retinal examination, flank pain or enlarged kidneys, palpable purpura (bruising), telangiectasias (extensive sclerosis), pallor, skin exoriations, muscle wasting, asterixis, myoclonic jerks, altered mental status, and pericardial rub.


In [33]:
print(rag_chain.invoke("What do you think of the current developments in Artificial Kidney?")['text'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Artificial kidneys, also known as bioartificial kidneys, are devices that aim to replicate the functions of a natural kidney in patients with kidney disease. The current developments in this field are promising, with researchers and scientists working towards creating safe and effective alternatives to traditional dialysis and transplantation methods.

One of the most notable developments in this area is the creation of a bioreactor with kidney cells that can perform critical functions such as balancing the body's fluids and releasing hormones that help regulate blood pressure. This technology has shown promise in animal tests, with successful implantation of the device in pigs.

However, despite these advancements, funding for research on artificial kidneys remains limited, which is holding up human clinical trials. According to Susan Merrell/UCSF, there are currently 93,000 people on the national waitlist for a kidney transplant, highlighting the urgent need for alternative options.

In [34]:
question_set = [
    "What kind of diet would you prefer for a CKD patient?",
    "Describe what exactly happens in CKD.",
    "How long do you think we are from a fully working artificial kidney?",
    "Give me a gist of recent advances in the treatment of kidney disease.",
    "How long does a transplant patient expected to live?"
]

In [36]:
for question in question_set:
    print(f"Question: {question}\nAnswer:{rag_chain.invoke(question)['text']}\n\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: What kind of diet would you prefer for a CKD patient?
Answer:
According to the information provided, a low-protein diet is recommended for CKD patients. However, it's important to note that the benefits of protein restriction must be balanced with the concern of precipitating malnutrition and/or protein wasting syndrome. Additionally, lower dietary acid loads (such as more fruits and vegetables and less meats, eggs, and cheeses) may also help protect against kidney injury. It's important to consult with a healthcare professional before making any significant changes to your diet.




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: Give me a gist of recent advances in the treatment of kidney disease.
Answer: Recent advances in the treatment of kidney disease include the successful testing of a prototype bioreactor in large animals, which aims to provide an alternative to dialysis for patients with end-stage renal disease (ESRD). Additionally, there are ongoing efforts to develop bioartificial kidneys through xenotransplants, which involve transplanting a kidney from a donor into a patient. These advancements offer hope for patients with chronic kidney disease (CKD) who may not be able to receive a human organ donation.




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Question: How long does a transplant patient expected to live?
Answer:
According to the article, the survival period for a group of cynomolgus monkeys that received xenotransplants (transplants from animals to humans) was between 82 to 642 days. However, it's important to note that the survival rate may vary depending on several factors such as the type of transplant, the health status of the recipient, and the quality of care provided. Additionally, the article mentions that only data from recipients surviving greater than 60 days were included in the analyses, which could affect the overall survival rate.


